In [24]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
# id=str(sys.argv[1])
id='82'
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
a_lite='#b4c7ed'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)

writer = pd.ExcelWriter('storage/report/consecutivo'+str(id)+'.xlsx', engine='xlsxwriter')

workbook = writer.book
##FORMATOS PARA EL TITULO------------------------------------------------------------------------------
rojo_l = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    #'fg_color': 'yellow',
    'font_color': 'red',
    'font_size':16})
negro_s = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12})
negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13,
    
    'text_wrap': True,
    'num_format': 'dd/mm/yyyy'}) 
rojo_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'red',
    'font_size':13})      

#FORMATOS PARA CABECERAS DE TABLA --------------------------------
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'center',
    'fg_color': 'yellow',
    'border': 1,})

blue_header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})
blue_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'num_format': '[$$-409]#,##0.00',
    'border': 1,
    'font_size':13})
blue_footer_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00',
    'font_size':11})
red_header_format = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})

red_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'font_size':13})


#FORMATOS PARA TABLAS PER CE------------------------------------

blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
blue_content_unit = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '0.00'})
blue_content_bold = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})
blue_content_date = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':9,
    'border_color':a_color,
    'num_format': 'dd/mm/yyyy'})
#FOOTER FORMATS---------------------------------------------------------
observaciones_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#BDD7EE',
    'border': 1})

blue_content_dll = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})

total_cereza_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#F4B084',
    'border': 1})
df=pd.DataFrame()
df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)

worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------

import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")

#Conectarse a la base
#traer datos


cotizaciones=pd.read_sql('select quotations.*, customers.customer,users.name from (quotations inner join customers on quotations.customer_id = customers.id) inner join users on users.id = quotations.user_id ',cnx)


worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------
worksheet.merge_range('B2:F2', 'REPORTE POR COTIZACION ', negro_b)
worksheet.merge_range('B3:F3', 'ADMINISTRATIVO', negro_s)
worksheet.merge_range('B4:F4', 'COSTOS ', negro_b)
worksheet.write('H2', 'AÑO', negro_b)

worksheet.write('I2', year, negro_b)
worksheet.merge_range('J2:K3', """FECHA DEL REPORTE
DD/MM/AAAA""", negro_b)

worksheet.write('L2', date, negro_b)
worksheet.insert_image("A1", "img/logo/logo.png",{"x_scale": 0.6, "y_scale": 0.6})


worksheet.merge_range('B6:B8', 'NO', blue_header_format)
worksheet.merge_range('C6:C8', 'DIAS', blue_header_format)
worksheet.merge_range('D6:D8', '# COT', blue_header_format)
worksheet.merge_range('E6:E8', "F. ENTREGA", blue_header_format)
worksheet.merge_range('F6:F8', 'VENDEDOR', blue_header_format)
worksheet.merge_range('G6:G8', 'CLIENTE', blue_header_format)
worksheet.merge_range('H6:H8', 'PRIO', blue_header_format)
worksheet.merge_range('I6:I8', "MONTO", blue_header_format)
worksheet.merge_range('J6:J8', 'MONTO MN', blue_header_format)
worksheet.merge_range('K6:K8', 'DESCRIPCION', blue_header_format)
worksheet.merge_range('L6:L8', 'KILOS', blue_header_format)
worksheet.merge_range('M6:M8', "ENCARGADO", blue_header_format)
worksheet.merge_range('N6:N8', 'OBSERVACIONES', blue_header_format)

price_cols=['price','total_price','import','unit_price']

aceros=pd.read_sql('select * from steels ',cnx)
aceros.loc[aceros['caliber']=='EST 3 IN','caliber']='EST3'

def cociente(a,b):
    if(b>0):
        return a/b
    else: 
        return 0
import tablas_dict
for j in range(len(cotizaciones)):
    products=pd.DataFrame()
    for i in tablas_dict.tablas:
        # print(i)
        #buscar en la base de datos todos los productos de esta tabla
        #pertenecientes a la cotizacion pedida por el usuario.
        p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
        p=p.assign(tabla=i)
        if(('cost' not in p.columns)&(len(p)>0)):
            if('caliber' not in p.columns):
                #esto es en especifico por un caso en que todas kas piezas son cal 14
                p=p.assign(caliber='14')
            try:
                p['caliber']=p['caliber'].str.replace('-','')
            except:
                print(' ')
            # print(str(p['caliber'].values[0]))
            costo=aceros.loc[aceros['caliber']==str(p['caliber'].values[0]),'cost'].values[0]
            if('total_kg' in p.columns):
                p=p.assign(cost=costo*p.total_kg)
            if('total_weight' in p.columns):
            
                p=p.assign(cost=costo*p.total_weight)
            if('weight_kg' in p.columns):
            
                p=p.assign(cost=costo*p.weight_kg)
            if('weight' in p.columns):
            
                p=p.assign(cost=costo*p.weight)
            if('long' in p.columns):
            
                p=p.assign(cost=costo*p.long)
            # print(i)
        products=products.append(p,ignore_index=True)
    cols_to_fill_str=['description','protector','model','sku']
    products[cols_to_fill_str]=products[cols_to_fill_str].fillna('')
    cols_kg=['weight','total_kg','total_weight','weight_kg']
    cols_m2=['m2','total_m2']
    price_cols=['price','total_price','import','unit_price']
    products[cols_kg+cols_m2]=products[cols_kg+cols_m2].fillna(0)

    kilos=products[cols_kg].sum(axis=1, numeric_only=True).sum()
    #trayendo informacion de materiales
    materials=pd.read_sql('select * from (materials left join price_list_screws on materials.price_list_screw_id= price_list_screws.id)left join price_lists on price_lists.id=materials.price_list_id',cnx)
    materials['type']=materials['type'].fillna('')
    materials=materials.rename(columns={'product':'tabla'})
    #calculando costos
    used_materials=products[['tabla','quotation_id']].merge(materials,how='inner',on='tabla')

    precio_pintura=pd.read_sql("select cost from price_lists where description like 'PINTURA'",cnx).values[0]
    costo_pintura=products.loc[products['caliber'].notna(),'cost'].sum()
    costo_total=costo_pintura+products['cost'].sum()+used_materials['cost'].fillna(0).sum(axis=1, numeric_only=True).sum()
    precio_venta=products[price_cols].sum(axis=1,numeric_only=True).sum()
    # print(used_materials)
    worksheet.write('B'+str(j+9),str(j+1),blue_content)
    worksheet.write('C'+str(j+9),' ',blue_content)
    worksheet.write('D'+str(j+9),cotizaciones['invoice'].values[j],blue_content)
    worksheet.write('E'+str(j+9),' ',blue_content)
    worksheet.write('F'+str(j+9),cotizaciones['customer'].values[j],blue_content)
    worksheet.write('G'+str(j+9),cotizaciones['name'].values[j],blue_content)
    worksheet.write('H'+str(j+9),'',blue_content)
    worksheet.write('I'+str(j+9),precio_venta*20,blue_content)
    worksheet.write('J'+str(j+9),precio_venta,blue_content)
    worksheet.write('K'+str(j+9),cotizaciones['system'].values[j],blue_content)
    worksheet.write('L'+str(j+9),kilos,blue_content_unit)
    worksheet.write('M'+str(j+9),'',blue_content)
    worksheet.write('N'+str(j+9),'',blue_content)
#ajustar columnas
worksheet.set_column('A:A',15)
worksheet.set_column('D:D',20)
worksheet.set_column('F:F',25)
worksheet.set_column('L:L',15)
worksheet.set_column('G:G',15)
worksheet.set_column('H:H',15)
worksheet.set_column('I:N',15)
worksheet.set_column('P:T',15)

#worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 1)  
workbook.close()


C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:216: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cotizaciones=pd.read_sql('select quotations.*, customers.customer,users.name from (quotations inner join customers on quotations.customer_id = customers.id) inner join users on users.id = quotations.user_id ',cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:250: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aceros=pd.read_sql('select * from steels ',cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Othe

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\L

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\L

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\L

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connectio

...

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\L

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\L

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\L

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\L

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:292: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2357973462.py:265: UserWarning: pandas only supports SQLAlchemy connectable (engine/connectio

In [12]:

price_cols=['price','total_price','import','unit_price']

aceros=pd.read_sql('select * from steels ',cnx)
aceros.loc[aceros['caliber']=='EST 3 IN','caliber']='EST3'

def cociente(a,b):
    if(b>0):
        return a/b
    else: 
        return 0
import tablas_dict
for j in range(len(cotizaciones)):
    products=pd.DataFrame()
    for i in tablas_dict.tablas:
        # print(i)
        #buscar en la base de datos todos los productos de esta tabla
        #pertenecientes a la cotizacion pedida por el usuario.
        p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
        p=p.assign(tabla=i)
        if(('cost' not in p.columns)&(len(p)>0)):
            if('caliber' not in p.columns):
                #esto es en especifico por un caso en que todas kas piezas son cal 14
                p=p.assign(caliber='14')
            try:
                p['caliber']=p['caliber'].str.replace('-','')
            except:
                print(' ')
            # print(str(p['caliber'].values[0]))
            costo=aceros.loc[aceros['caliber']==str(p['caliber'].values[0]),'cost'].values[0]
            if('total_kg' in p.columns):
                p=p.assign(cost=costo*p.total_kg)
            if('total_weight' in p.columns):
            
                p=p.assign(cost=costo*p.total_weight)
            if('weight_kg' in p.columns):
            
                p=p.assign(cost=costo*p.weight_kg)
            if('weight' in p.columns):
            
                p=p.assign(cost=costo*p.weight)
            if('long' in p.columns):
            
                p=p.assign(cost=costo*p.long)
            # print(i)
        products=products.append(p,ignore_index=True)
    cols_to_fill_str=['description','protector','model','sku']
    products[cols_to_fill_str]=products[cols_to_fill_str].fillna('')
    cols_kg=['weight','total_kg','total_weight','weight_kg']
    cols_m2=['m2','total_m2']
    price_cols=['price','total_price','import','unit_price']
    products[cols_kg+cols_m2]=products[cols_kg+cols_m2].fillna(0)


    #trayendo informacion de materiales
    materials=pd.read_sql('select * from (materials left join price_list_screws on materials.price_list_screw_id= price_list_screws.id)left join price_lists on price_lists.id=materials.price_list_id',cnx)
    materials['type']=materials['type'].fillna('')
    materials=materials.rename(columns={'product':'tabla'})
    #calculando costos
    used_materials=products[['tabla','quotation_id']].merge(materials,how='inner',on='tabla')

    precio_pintura=pd.read_sql("select cost from price_lists where description like 'PINTURA'",cnx).values[0]
    costo_pintura=products.loc[products['caliber'].notna(),'cost'].sum()
    costo_total=costo_pintura+products['cost'].sum()+used_materials['cost'].fillna(0).sum(axis=1, numeric_only=True).sum()
    precio_venta=products[price_cols].sum(axis=1,numeric_only=True).sum()
    # print(used_materials)

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aceros=pd.read_sql('select * from steels ',cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarni

double_deep_crossbars
double_deep_floors
double_deep_floor_reinforcements
double_deep_heavy_load_frames
14
double_deep_heavy_load_frames
double_deep_joist_box25s
double_deep_joist_box25_caliber14s
double_deep_joist_box2s
double_deep_joist_box2_caliber14s
double_deep_joist_c2_s
double_deep_joist_chairs
double_deep_joist_l25_caliber14s
double_deep_joist_l25_s
double_deep_joist_l2_caliber14s
double_deep_joist_l2_s
double_deep_joist_lrs
double_deep_joist_structurals
double_deep_miniature_frames


C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Loca

double_deep_spacers
double_deep_structural_frames
freights
grills
installations
packagings
quot25_j_galvanized_panels
quot25_j_painted_panels
quot2_j_galvanized_panels
quot2_j_painted_panels
 
14
quot2_j_painted_panels
quotation_administratives
quotation_installs
quotation_protectors
quotation_specials

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Loca

...

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) o

selective_joist_box2s
10
selective_joist_box2s
selective_joist_box2_caliber14s
14
selective_joist_box2_caliber14s
selective_joist_c2_s
10
selective_joist_c2_s
selective_joist_chairs
selective_joist_l25_caliber14s
14
selective_joist_l25_caliber14s
selective_joist_l25_s
10
selective_joist_l25_s
selective_joist_l2_caliber14s
14
selective_joist_l2_caliber14s
selective_joist_l2_s
10
selective_joist_l2_s
selective_joist_lrs

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(cotizaciones['id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) o


selective_joist_structurals
EST3
selective_joist_structurals
selective_miniature_frames
14
selective_miniature_frames
selective_spacers
12
selective_spacers
selective_structural_frames
EST3
selective_structural_frames
uninstalls
wood
                              tabla quotation_id  id  price_list_screw_id  \
0                  selective_floors           52  56                  1.0   
1       selective_heavy_load_frames           52   1                  1.0   
2       selective_heavy_load_frames           52   2                  8.0   
3       selective_heavy_load_frames           52   3                  NaN   
4             selective_joist_box2s           52  11                  9.0   
5   selective_joist_box2_caliber14s           52  10                  9.0   
6              selective_joist_c2_s           52  15                  9.0   
7    selective_joist_l25_caliber14s           52   6                  9.0   
8             selective_joist_l25_s           52   7                  9.

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  materials=pd.read_sql('select * from (materials left join price_list_screws on materials.price_list_screw_id= price_list_screws.id)left join price_lists on price_lists.id=materials.price_list_id',cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_10944\2781305644.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  precio_pintura=pd.read_sql("select cost from price_lists where description like 'PINTURA'",cnx).values[0]


In [19]:
cotizaciones=pd.read_sql('select quotations.*, customers.customer,users.name from (quotations inner join customers on quotations.customer_id = customers.id) inner join users on users.id = quotations.user_id ',cnx)
cotizaciones

C:\Users\hp\AppData\Local\Temp\ipykernel_10944\428819625.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cotizaciones=pd.read_sql('select quotations.*, customers.customer,users.name from (quotations inner join customers on quotations.customer_id = customers.id) inner join users on users.id = quotations.user_id ',cnx)


,id,invoice,system,customer_id,type,user_id,system_price,installation_price,transfer_price,status,created_at,updated_at,customer,name
0,52,1051,SISTEMAS DE ALMACENAMIENTO (RACKS),1,SELECTIVO,1,6779.42,0.0,3094.38,Iniciada,2023-03-24 15:04:14,2023-09-22 13:09:48,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario
1,53,1052,SISTEMAS DE ALMACENAMIENTO (RACKS),1,DOBLE PROFUNDIDAD,1,0.00,0.0,0.00,Iniciada,2023-03-25 05:07:09,2023-06-27 09:50:36,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario
2,54,1053,SISTEMAS DE ALMACENAMIENTO (RACKS),1,SELECTIVO,1,NaN,NaN,NaN,Iniciada,2023-03-25 05:11:10,2023-03-25 05:12:34,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario
3,55,1054,SISTEMAS DE ALMACENAMIENTO (RACKS),1,SELECTIVO,1,0.00,0.0,0.00,Iniciada,2023-03-25 05:24:06,2023-03-25 10:01:25,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario
4,56,1055,SISTEMAS DE ALMACENAMIENTO (RACKS),1,SELECTIVO,1,63767.88,0.0,34.20,Iniciada,2023-04-03 16:04:06,2023-04-03 16:37:34,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario
5,57,1056,SISTEMAS DE ALMACENAMIENTO (RACKS),1,SELECTIVO,1,NaN,NaN,NaN,Iniciada,2023-04-09 08:54:47,2023-04-09 08:56:23,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario
6,58,1057,SISTEMAS DE ALMACENAMIENTO (RACKS),1,SELECTIVO,1,NaN,NaN,NaN,Iniciada,2023-04-09 09:39:24,2023-04-09 09:40:38,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario
7,59,1058,SISTEMAS DE ALMACENAMIENTO (RACKS),1,SELECTIVO,1,NaN,NaN,NaN,Iniciada,2023-04-10 05:14:58,2023-04-10 05:15:57,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario
8,60,1059,SISTEMAS DE ALMACENAMIENTO (RACKS),1,DOBLE PROFUNDIDAD,1,15024.71,0.0,13.68,Iniciada,2023-04-10 07:06:59,2023-04-10 07:11:41,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario
9,61,1060,SISTEMAS DE ALMACENAMIENTO (RACKS),1,SELECTIVO,1,16613.03,0.0,58.14,Iniciada,2023-04-10 12:16:42,2023-04-10 12:19:33,BECTON DICKINSON DE MEXICO S. A. DE C. V,Super Usuario


In [21]:
kilos=products[cols_kg].sum(axis=1, numeric_only=True).sum()

In [23]:
products[cols_kg]

,weight,total_kg,total_weight,weight_kg
0,0.0000,3781.95,0.0,0.0000
1,0.0000,0.00,0.0,0.0000
2,0.0000,0.00,0.0,0.0000
3,0.0000,0.00,0.0,0.0000
4,0.0000,0.00,0.0,0.0000
5,1.9000,0.00,1.9,0.0000
6,0.0000,0.00,0.0,0.0000
7,0.0000,0.00,718.0,0.0000
8,0.0000,0.00,0.0,0.0000
9,0.0000,0.00,0.0,0.0000


In [15]:
len(materials)

33

In [3]:
330*0.05

16.5

In [ ]:
a*18.92=b*16.5

In [7]:
a=6
a*18.92/16.5

6.880000000000001